In [19]:
import sys
import numpy as np
import import_ipynb
from pathlib import Path
from scipy.sparse.linalg import spsolve

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from pre_processing import create_domain, read_mesh
    from fem_processing import boundary_conditions, matrices_assembly
    from pos_processing import graph_results
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Modules imports were successful!


# Project 1: Coaxial Cable

Considere um cabo coaxial com dois dielétricos apresentado na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/coaxial_problem.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of Coaxial Problem.</figcaption>
</figure>

O raio do condutor interno é ``a`` e do condutor externo é ``b``. O primeiro dielétrico, $\varepsilon_{1}$, preenche o anel circular entre os raios ``a`` e ``c``, enquanto o segundo dielétrico, $\varepsilon_{2}$, preenche o anel entre os raios ``c`` e ``b``.  

Entre o condutor interno e o externo é aplicada uma diferença de potencial igual a $V$.  

Seja $V = 1$ (potencial zero em ``a`` e um em ``b``), $a = 2 \, mm$, $b = 8 \, mm$, $c = 5 \, mm$, $\varepsilon_{r1} = 2$ e $\varepsilon_{r2} = 4$. Para este problema, além da convergência nas normas $L_2$ e da energia, calcule a convergência para o valor da capacitância do dispositivo.

# 3-noded Linear Triangular Elements, $P_1$
## Pre-processor module
### Geometry and mesh Domain

In [20]:
FINITE_ELEMENT = ("Triangle", 1)

BOUNDARY = [{'type': 'Dirichlet', 'tag': 101, 'value': 0.0, 'name': 'inner_conductor'},
            {'type': 'Dirichlet', 'tag': 102, 'value': 1.0, 'name': 'outer_conductor'}]

MATERIAL = [{'type': 'relative_electric_permittivity', 'tag': 201, 'value': 2, 'name': 'medium_1'},
            {'type': 'relative_electric_permittivity', 'tag': 202, 'value': 4, 'name': 'medium_2'}]

# Create mesh from file geometry
create_domain.coaxial_geometry(FINITE_ELEMENT, BOUNDARY, MATERIAL, h=1E-2, view_mesh=True)
mesh_data = read_mesh.get_data(FINITE_ELEMENT, BOUNDARY, MATERIAL, model='coaxial', info_mode=False)

Model coaxial_domain_Triangle1 (2D)
Info     : 8 geometric entities
Info     : 4 Physical Groups
Info     : 26 nodes in total
Info     : 38 2-D elements in total


### Mesh Data Dictionaries

In [21]:
mesh_nodes = mesh_data['nodes_data']; mesh_nodes[:1]

[{'TagID': 1,
  'global_coord': (0.002, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}}]

In [22]:
conn_data = mesh_data['conn_data']; conn_data[:1]

[{'TagID': 25,
  'conn_list': [14, 25, 8],
  'material': {'type': 'relative_electric_permittivity',
   'tag': 201,
   'value': 2,
   'name': 'medium_1'}}]

### Figure 1: Coaxial meshed domain 

In [23]:
graph_results.plot_mesh(mesh_data, FINITE_ELEMENT, model='coaxial', Numbering=True)

<string>:38: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.


Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pre_processing\pictures\coaxial_meshed_domain_Triangle1.svg


<figure>
    <img src="pre_processing/pictures/coaxial_meshed_domain_Triangle1.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Coaxial meshed domain.</figcaption>
</figure>

In [24]:
nodes = [node['global_coord'] for node in mesh_data['nodes_data']]
conn = [element['conn_list'] for element in mesh_data['conn_data']]

### Assembly Matrices
#### `assembly.global_nodes_coordinates()` 

In [25]:
ai, xi, yi = matrices_assembly.global_nodes_coordinates(0, mesh_data)
print(f"'a_1' Global coordinates: {ai}")
print(f"'x_1' x-coordinates: {xi}")
print(f"'y_1' y-coordinates: {yi}")

'a_1' Global coordinates: [(-0.001112604669781573, -0.004874639560909118), (0.0007016955188292377, -0.003074328940637243), (-0.0004450418679126153, -0.00194985582436365)]
'x_1' x-coordinates: [-0.001112604669781573, 0.0007016955188292377, -0.0004450418679126153]
'y_1' y-coordinates: [-0.004874639560909118, -0.003074328940637243, -0.00194985582436365]


#### `map_to_physical_coordinates()`

In [26]:
xi_master = (1, 0)
xg, yg = matrices_assembly.isomapping_to_global_coordinates(ai, xi_master, FINITE_ELEMENT)
print(f"'e_1' Master coordinates: {xi_master} --> Global coordinates: ({xg}, {yg})")

'e_1' Master coordinates: (1, 0) --> Global coordinates: (0.0007016955188292377, -0.003074328940637243)


#### `apply_physics()`

In [27]:
mesh_data = ps.apply_physics(mesh_data, FINITE_ELEMENT)
mesh_data['conn_data'][:1]

[{'TagID': 25,
  'conn_list': [14, 25, 8],
  'material': {'type': 'relative_electric_permittivity',
   'tag': 201,
   'value': 2,
   'name': 'medium_1',
   'a_value': 1.77083756256e-11},
  'source': {'type': 'free_source', 'value': 0}}]

#### Material Propriety, $a$

In [28]:
a = [element['material']['a_value'] for element in mesh_data['conn_data']]
print("Material 'a' values: \n", np.array(a))

Material 'a' values: 
 [1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 3.54167513e-11 3.54167513e-11 3.54167513e-11 3.54167513e-11
 3.54167513e-11 3.54167513e-11 3.54167513e-11 3.54167513e-11
 3.54167513e-11 3.54167513e-11 3.54167513e-11 3.54167513e-11
 3.54167513e-11 3.54167513e-11]


#### Source: Analytical source, $f(x)$

In [29]:
Fx = [element['source']['value'] for element in mesh_data['conn_data']]
print("Source vector Fx: \n", np.array(Fx))

Source vector Fx: 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]


#### Linear Jacobian

In [30]:
Je = matrices_assembly.jacobian(e=0, mesh_data=mesh_data, element_type=FINITE_ELEMENT, xik=(0, 0)) 
Jdet, Jinv = np.linalg.det(Je), np.linalg.inv(Je)
print("Jacobian matrix for element e_0:\n", Je)
print("Determinant of the Jacobian matrix for element e_0:", Jdet)

Jacobian matrix for element e_0:
 [[0.0018143  0.00180031]
 [0.00066756 0.00292478]]
Determinant of the Jacobian matrix for element e_0: 4.104615282957137e-06


#### Local Elements $e_1$

In [31]:
Ae, Fe, areae = matrices_assembly.linear_local_matrices(e=0, mesh_data=mesh_data)
print("Local stiffness matrix for element e_1:\n", Ae)
print("Local load vector for element e_1:\n", Fe)
print("Area of element e_1: ", areae)

Local stiffness matrix for element e_1:
 [[ 5.56420066e-12 -5.44313183e-12 -1.21068830e-13]
 [-5.44313183e-12  1.94141679e-11 -1.39710361e-11]
 [-1.21068830e-13 -1.39710361e-11  1.40921049e-11]]
Local load vector for element e_1:
 [[0.]
 [0.]
 [0.]]
Area of element e_1:  2.0523076414785684e-06


#### Global “stiffness” matrix, $A_g$

In [32]:
Ag, fg = matrices_assembly.get_global_matrix(mesh_data, FINITE_ELEMENT)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (26, 26)


### Imposition of Boundary Conditions

In [ ]:
dirichlet_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
dirichlet_values = [
    node['bc']['value'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
free_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Free']

print(f"The entire domain has {len(nodes)} nodes: " 
      f"{len(nodes) - len(dirichlet_nodes)} free nodes and {len(dirichlet_nodes)} Dirichlet nodes.")
print("Free nodes: \n", np.array(free_nodes))
print("Dirichlet nodes: \n", np.array(dirichlet_nodes))
print("Dirichlet nodes values: \n", np.array(dirichlet_values))

The entire domain has 26 nodes: 12 free nodes and 14 Dirichlet nodes.
Free nodes: 
 [ 2 10 11 12 13 14 15 22 23 24 25 26]
Dirichlet nodes: 
 [ 1  3  4  5  6  7  8  9 16 17 18 19 20 21]
Dirichlet nodes values: 
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]


### Mapping Global nodes to reduced system

This code creates a dictionary called ``global_to_reduced``, which maps the global indices of the mesh nodes to the reduced indices, i.e. the indices that correspond only to the nodes that are not in the _Dirichlet boundary conditions_.

In [34]:
global_to_reduced = {node: i for i, node in enumerate(
        idx_n for idx_n in range(1, len(nodes)+1) if idx_n not in dirichlet_nodes
    )}
print("Global to reduced nodes mapping:"); global_to_reduced

Global to reduced nodes mapping:


{2: 0,
 10: 1,
 11: 2,
 12: 3,
 13: 4,
 14: 5,
 15: 6,
 22: 7,
 23: 8,
 24: 9,
 25: 10,
 26: 11}

## Processor Module
### Reduced Global matrix $A_{gr}$ with boundary conditions

In [35]:
Agr, fgr = boundary_conditions.get_reduced_global_matrix(mesh_data, FINITE_ELEMENT)
print("Reduced global matrix shape:", Agr.shape)
print("Reduced global matrix: \n", Agr)
print("Reduced force vector: \n", fgr)

Reduced global matrix shape: (12, 12)
Reduced global matrix: 
   (0, 0)	1.4605304934885143e-10
  (0, 1)	-1.7769121477991374e-11
  (0, 6)	-2.464882842781293e-11
  (0, 7)	-1.9656305570919502e-11
  (1, 0)	-1.776912147799138e-11
  (1, 1)	1.490014951844895e-10
  (1, 2)	-1.7769121477991335e-11
  (1, 7)	-1.9656305570919512e-11
  (1, 8)	-1.965630557091949e-11
  (2, 1)	-1.7769121477991335e-11
  (2, 2)	1.4900149518448945e-10
  (2, 3)	-1.7769121477991258e-11
  (2, 8)	-1.9656305570919522e-11
  (2, 9)	-1.9656305570919483e-11
  (3, 2)	-1.7769121477991258e-11
  (3, 3)	1.490014951844892e-10
  (3, 4)	-1.7769121477991125e-11
  (3, 9)	-1.9656305570919512e-11
  (3, 11)	-1.965630557091945e-11
  (4, 3)	-1.7769121477991122e-11
  (4, 4)	1.4605304934885117e-10
  (4, 5)	-2.4648828427812826e-11
  (4, 11)	-1.9656305570919564e-11
  (5, 4)	-2.4648828427812826e-11
  (5, 5)	1.4605304934885073e-10
  (5, 6)	-1.776912147799085e-11
  (5, 10)	-1.965630557091945e-11
  (6, 0)	-2.4648828427812923e-11
  (6, 5)	-1.776912147799

### Global Potential Vector

In [36]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.get_global_potentials(mesh_data, ur)
print("The reduced solution vector ur is: \n", ur)
print("The global potential vector V is: \n", potential_u)

The reduced solution vector ur is: 
 [0.78787879 0.78787879 0.78787879 0.78787879 0.78787879 0.78787879
 0.78787879 0.39393939 0.39393939 0.39393939 0.39393939 0.39393939]
The global potential vector V is: 
 [0.         0.78787879 1.         0.         0.         0.
 0.         0.         0.         0.78787879 0.78787879 0.78787879
 0.78787879 0.78787879 0.78787879 1.         1.         1.
 1.         1.         1.         0.39393939 0.39393939 0.39393939
 0.39393939 0.39393939]


### Solution at each element

In [37]:
u_list = []
for e in range(len(conn)):
    uh_values = [potential_u[id - 1] for id in conn[e]]
    u_list.append(uh_values)
print("The solution u for Node Element is: \n", np.array(u_list))

The solution u for Node Element is: 
 [[0.78787879 0.39393939 0.        ]
 [0.78787879 0.39393939 0.        ]
 [0.         0.39393939 0.78787879]
 [0.         0.39393939 0.78787879]
 [0.78787879 0.39393939 0.        ]
 [0.         0.39393939 0.78787879]
 [0.         0.39393939 0.78787879]
 [0.         0.39393939 0.78787879]
 [0.78787879 0.39393939 0.        ]
 [0.78787879 0.39393939 0.        ]
 [0.         0.39393939 0.        ]
 [0.         0.39393939 0.        ]
 [0.         0.39393939 0.        ]
 [0.         0.39393939 0.        ]
 [0.         0.39393939 0.        ]
 [0.78787879 0.39393939 0.78787879]
 [0.78787879 0.39393939 0.78787879]
 [0.78787879 0.39393939 0.78787879]
 [0.78787879 0.39393939 0.78787879]
 [0.78787879 0.39393939 0.78787879]
 [0.78787879 0.         0.        ]
 [0.         0.78787879 0.78787879]
 [0.78787879 0.         0.        ]
 [0.78787879 0.         0.78787879]
 [1.         0.78787879 0.78787879]
 [1.         0.78787879 1.        ]
 [1.         0.78787879 0.

## Post-Processor

In [38]:
graph_results.approximate_fem_solution(FINITE_ELEMENT, mesh_data, potential_u, model='coaxial')
graph_results.coaxial_analytical_solution()

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pos_processing\pictures\coaxial_domain_solution_Triangle1.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pos_processing\pictures\coaxial_analytical_solution.svg


### Figure 2: Scalar Field Distribution on a Triangular Mesh

<figure>
    <img src="pos_processing/pictures/coaxial_domain_solution_Triangle1.svg" alt="Fig.2" style="width:100%;" />
    <figcaption>Figure 2: Scalar Field Distribution on a Triangular Mesh.</figcaption>
</figure>

### Figure 3: Analytical Solution for a Scalar Field Distribution

<figure>
    <img src="pos_processing/pictures/coaxial_analytical_solution.svg" alt="Fig.3" style="width:100%;" />
    <figcaption>Figure 3: Analytical Solution for a Scalar Field Distribution.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.